In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook', style = 'whitegrid', font_scale = 1.3)

In [2]:
sys.path.append('..')
from modules.plotting_metrics import PlotMetric

In [19]:
prot_name = 'cdk2'
# Este es el DF que no tiene aun afgregados los valores AUC de las conformaciones
path_to_json_file = glob.glob((os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys_LIGS_INFO_LABELS.json')))[0]
                      
df_prot = pd.read_json(path_to_json_file)

In [20]:
# Extract just the interesting features
df_prot.columns
columns_ = ['Labels_conf', 'Inhib', 'Inhib_mass']
df_CDK2 = df_prot[columns_]

#### MDS info

In [4]:
# Read the MDS resutls
import pickle

# Pocket MDS
path_mds_obj_pk = os.path.join('..', 'data', 'trajectory_analysis', 'cMDS_Pocket_402_obj.pyobj')
with open(path_mds_obj_pk, 'rb') as f:
    mds_pocket_402 = pickle.load(f)
    
# Pisani MDS
path_mds_obj = os.path.join('..', 'data', 'trajectory_analysis', 'cMDS_Pisani_402_obj.pyobj')
with open(path_mds_obj, 'rb') as f:
    mds_pisani_402 = pickle.load(f)
    
a = mds_pisani_402[0][:2]
b = mds_pocket_402[0][:2]

c = np.concatenate((a, b))

np.savetxt('./mds_subspaces_pis_pk.csv', c, delimiter = ',')

In [6]:
# Helper function
%run ./helper_functions_S6.ipynb

In [15]:
score_type = 'Dk_lef'
df_swarm_dklef = pd.DataFrame()

for db_name in scores_dic:
    _X = scores_dic[db_name][score_type]['X']
    _y = scores_dic[db_name][score_type]['y']
    auc_scores = _X.apply(lambda x: roc_auc_score(y_true =_y, 
                                  y_score = -1 * x), axis = 0)
    score_type_ = score_type.lower().replace('_', '')
    df_swarm_dklef[db_name + '_vrd_' + score_type_] = auc_scores

score_type = 'Dk_sc'
df_swarm_dksc = pd.DataFrame()

for db_name in scores_dic:
    _X = scores_dic[db_name][score_type]['X']
    _y = scores_dic[db_name][score_type]['y']
    auc_scores = _X.apply(lambda x: roc_auc_score(y_true =_y, 
                                  y_score = -1 * x), axis = 0)
    score_type_ = score_type.lower().replace('_', '')
    df_swarm_dksc[db_name + '_vrd_' + score_type_] = auc_scores
    

In [25]:
x = pd.concat([df_CDK2, df_swarm_dksc, df_swarm_dklef], axis = 1)

In [26]:
x.to_csv('./df_pdb_cdk2_features_and_scores.csv')

### Consensus Scorings

In [ ]:
# kmeans or random
# Consensus type
# Docking score
# Database

# Save each file

In [1]:
from glob import glob
import pickle
import numpy as np
import pandas as pd

In [2]:
path_ = '../data/ml_evaluations/consensus_scoring/'

# Each final csv goint to have the 
# consensus values, and databases



In [65]:
path_ = '../data/ml_evaluations/'
file = path_ + 'DUD_CSAR_LogRg_random_Dksc.obj'
with open(file, 'rb') as f:
            dict_auc = pickle.load(f)
        
dict_auc['DUD'].mean(axis = 0)

1      0.577033
2      0.000000
3      0.609900
4      0.000000
5      0.615933
         ...   
398    0.000000
399    0.788467
400    0.000000
401    0.788667
402    0.000000
Length: 402, dtype: float64

In [57]:
path_ = '../data/ml_evaluations/consensus_scoring/'
file = path_ + 'CSAR_DUD_DEKOIS_random_Cons_Rbn.obj'
with open(file, 'rb') as f:
            dict_auc = pickle.load(f)
        
dict_auc.keys()

dict_keys(['DUD_Dksc_Rdm_Cons_Rbn', 'DEKOIS_Dksc_Rdm_Cons_Rbn', 'CSAR_Dksc_random_Cons_Rbn', 'CSAR_Dklef_random_Cons_Rbn', 'DUD_Dksc_random_Cons_Rbn', 'DUD_Dklef_random_Cons_Rbn', 'DEKOIS_Dksc_random_Cons_Rbn', 'DEKOIS_Dklef_random_Cons_Rbn'])

In [58]:
path_ = '../data/ml_evaluations/consensus_scoring/'

methods = ('kmeans', 'rfe', 'correlated', 'random')
cons_list_names = ['Rbn', 'Rbs', 'Rbr', 'Rbv_*2', 'Rexp*2']
inner_keys = ['pisani', 'pocket']

final_list = []

for method in methods:
    for cons in cons_list_names:
        file = glob(F'{path_}*{method}*{cons}*')[0]
        print(file)

        with open(file, 'rb') as f:
            dict_auc = pickle.load(f)
        # iterate over the keys
        for key in dict_auc.keys():
            # for kmeans we currently have pisani and pocket
            for in_key in inner_keys:
                if method == 'kmeans':
                    _in_key = '-' + in_key.lower()
                    list_values = dict_auc[key][in_key].tolist()
                elif method == 'rfe':
                    _in_key = ''
                    list_values = dict_auc[key]['rfe'].tolist()
                elif method == 'correlated':
                    _in_key = ''
                    list_values = dict_auc[key]['correlated'].tolist()
                elif method == 'random':
                    _in_key = ''
                    if '_Rdm_' in key:
                        break
                    list_values = dict_auc[key].mean(axis = 0).tolist()  
                print(key)    
                names = key.split('_')
                # Get the databaseName
                db = names[0]
                # Get the score type
                score = names[1]

                # Append new elemnts at the begining of the list
                # Scoring > MethodFeature > DB > Consensus > Values
                list_values.insert(0, cons.lower())
                list_values.insert(0, db.lower())
                list_values.insert(0, (method + _in_key).lower())
                list_values.insert(0, score.lower())
                list_values.insert(0, 'consensus') # Empty for Consensus
                final_list.append( list_values )
                if method == 'kmeans': continue
                else: break
# Save the file
file_name = './cosensus_scorings.csv'
df_ = pd.DataFrame(final_list).round(3)
df_.to_csv(file_name)

../data/ml_evaluations/consensus_scoring/CSAR_DUD_DEKOIS_kmeans_Cons_Rbn.obj
CSAR_Dksc_kmeans_Cons_Rbn
CSAR_Dksc_kmeans_Cons_Rbn
CSAR_Dklef_kmeans_Cons_Rbn
CSAR_Dklef_kmeans_Cons_Rbn
DUD_Dksc_kmeans_Cons_Rbn
DUD_Dksc_kmeans_Cons_Rbn
DUD_Dklef_kmeans_Cons_Rbn
DUD_Dklef_kmeans_Cons_Rbn
DEKOIS_Dksc_kmeans_Cons_Rbn
DEKOIS_Dksc_kmeans_Cons_Rbn
DEKOIS_Dklef_kmeans_Cons_Rbn
DEKOIS_Dklef_kmeans_Cons_Rbn
../data/ml_evaluations/consensus_scoring/CSAR_DUD_DEKOIS_kmeans_Cons_Rbs.obj
CSAR_Dksc_kmeans_Cons_Rbs
CSAR_Dksc_kmeans_Cons_Rbs
CSAR_Dklef_kmeans_Cons_Rbs
CSAR_Dklef_kmeans_Cons_Rbs
DUD_Dksc_kmeans_Cons_Rbs
DUD_Dksc_kmeans_Cons_Rbs
DUD_Dklef_kmeans_Cons_Rbs
DUD_Dklef_kmeans_Cons_Rbs
DEKOIS_Dksc_kmeans_Cons_Rbs
DEKOIS_Dksc_kmeans_Cons_Rbs
DEKOIS_Dklef_kmeans_Cons_Rbs
DEKOIS_Dklef_kmeans_Cons_Rbs
../data/ml_evaluations/consensus_scoring/CSAR_DUD_DEKOIS_kmeans_Cons_Rbr.obj
CSAR_Dksc_kmeans_Cons_Rbr
CSAR_Dksc_kmeans_Cons_Rbr
CSAR_Dklef_kmeans_Cons_Rbr
CSAR_Dklef_kmeans_Cons_Rbr
DUD_Dksc_kmeans_Con

In [59]:
df_

,0,1,2,3,4,5,6,7,8,9,...,397,398,399,400,401,402,403,404,405,406
0,consensus,dksc,kmeans-pisani,csar,rbn,0.759,0.798,0.807,0.765,0.725,...,0.761,0.761,0.761,0.761,0.761,0.761,0.762,0.761,0.762,0.761
1,consensus,dksc,kmeans-pocket,csar,rbn,0.602,0.789,0.728,0.757,0.748,...,0.760,0.760,0.760,0.761,0.761,0.761,0.761,0.761,0.761,0.761
2,consensus,dklef,kmeans-pisani,csar,rbn,0.859,0.859,0.892,0.853,0.842,...,0.861,0.861,0.861,0.861,0.861,0.861,0.862,0.862,0.861,0.861
3,consensus,dklef,kmeans-pocket,csar,rbn,0.777,0.859,0.845,0.859,0.820,...,0.862,0.862,0.862,0.862,0.862,0.862,0.862,0.861,0.862,0.861
4,consensus,dksc,kmeans-pisani,dud,rbn,0.634,0.607,0.626,0.601,0.574,...,0.600,0.600,0.600,0.600,0.600,0.600,0.600,0.600,0.600,0.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,consensus,dksc,random,dekois,rexp*2,0.688,0.706,0.706,0.713,0.712,...,0.706,0.705,0.706,0.706,0.706,0.705,0.706,0.706,0.706,0.705
146,consensus,dksc,random,csar,rexp*2,0.732,0.722,0.709,0.703,0.696,...,0.692,0.692,0.692,0.693,0.692,0.692,0.693,0.692,0.692,0.692
147,consensus,dklef,random,csar,rexp*2,0.834,0.837,0.844,0.831,0.846,...,0.818,0.818,0.818,0.818,0.818,0.818,0.818,0.818,0.818,0.818
148,consensus,dklef,random,dud,rexp*2,0.682,0.680,0.690,0.694,0.688,...,0.676,0.676,0.676,0.675,0.676,0.675,0.676,0.675,0.675,0.675


In [74]:
path_ = '../data/ml_evaluations'

# DEKOIS_CSAR_LogRg_kmeans-pisani_Dksc

methods = ('kmeans-pisani', 'kmeans-pocket', 'rfe', 'correlated', 'random')
ml_list_names = ['LinearSVC', 'rbfSVC', 'LogRg']
score_types = ['Dksc', 'Dklef']

databases = {'CSAR': 'DUD_DEKOIS', 'DEKOIS': 'DUD_CSAR', 'DUD': 'DEKOIS_CSAR'}

final_list = []

for db, tests_names in databases.items():
    for method in methods:
        for ml in ml_list_names:
            for score in score_types:
                file_name = F'{path_}/{tests_names}_{ml}_{method}_{score}*'
                #print(file_name)
                file = glob(file_name)[0]
                with open(file, 'rb') as f:
                    dict_auc = pickle.load(f)
                # iterate over the keys
                for db_test in dict_auc.keys():
                    if method == 'random':
                        list_values = dict_auc[db_test].mean(axis = 0).round(4).tolist()
                        l = list_values[::2]
                        list_values = [x for pair in zip(l,l) for x in pair]
                    else:
                        list_values = dict_auc[db_test].values.tolist()[0]

                    # Append new elemnts at the begining of the list
                    # Scoring > MethodFeature > DB > TrainDB > ML Estimator > Values
                    list_values.insert(0, ml)
                    
                    list_values.insert(0, db_test.lower())
                    list_values.insert(0, method.lower())
                    list_values.insert(0, score.lower())
                    list_values.insert(0, db.lower())
                    final_list.append( list_values )
                    
file_name = './ml_scorings.csv'          
df_ml = pd.DataFrame(final_list).round(3)
df_ml.to_csv(file_name)

In [75]:
df_ml

,0,1,2,3,4,5,6,7,8,9,...,397,398,399,400,401,402,403,404,405,406
0,csar,dksc,kmeans-pisani,dud,LinearSVC,0.634,0.612,0.647,0.692,0.665,...,0.784,0.783,0.783,0.784,0.784,0.784,0.784,0.784,0.784,0.785
1,csar,dksc,kmeans-pisani,dekois,LinearSVC,0.701,0.715,0.713,0.724,0.613,...,0.785,0.786,0.787,0.787,0.786,0.787,0.787,0.786,0.786,0.786
2,csar,dklef,kmeans-pisani,dud,LinearSVC,0.720,0.701,0.747,0.765,0.707,...,0.764,0.764,0.764,0.765,0.764,0.765,0.764,0.765,0.766,0.767
3,csar,dklef,kmeans-pisani,dekois,LinearSVC,0.687,0.686,0.695,0.689,0.670,...,0.797,0.797,0.798,0.797,0.797,0.797,0.796,0.796,0.796,0.796
4,csar,dksc,kmeans-pisani,dud,rbfSVC,0.639,0.613,0.648,0.701,0.683,...,0.800,0.801,0.801,0.802,0.801,0.801,0.800,0.800,0.800,0.799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,dud,dklef,random,csar,rbfSVC,0.337,0.337,0.453,0.453,0.561,...,0.923,0.923,0.922,0.922,0.923,0.923,0.921,0.921,0.922,0.922
176,dud,dksc,random,dekois,LogRg,0.663,0.663,0.670,0.670,0.696,...,0.711,0.711,0.712,0.712,0.713,0.713,0.712,0.712,0.713,0.713
177,dud,dksc,random,csar,LogRg,0.704,0.704,0.706,0.706,0.766,...,0.918,0.918,0.919,0.919,0.919,0.919,0.920,0.920,0.920,0.920
178,dud,dklef,random,dekois,LogRg,0.661,0.661,0.682,0.682,0.688,...,0.750,0.750,0.751,0.751,0.751,0.751,0.751,0.751,0.751,0.751


In [73]:
x = [1, 2, 3, 4, 5, 6, 7, 8]
x[::2]

[1, 3, 5, 7]